# Generate frames from video

In [1]:
import numpy as np
import cv2
from pathlib import Path
from google.colab import drive
from tqdm import tqdm

In [2]:
drive_path = Path('/').absolute() / 'content' / 'drive'
drive.mount(str(drive_path))

video_name = 'NightSky.mp4'
datasets_path = drive_path / 'My Drive' / 'StyleGAN2-ADA' / 'datasets'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import video

In [3]:
video_path = datasets_path / 'source' / video_name
if video_path.is_file():
    video_cap = cv2.VideoCapture(str(video_path))
    video_width  = video_cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    video_height = video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    video_fps    = video_cap.get(cv2.CAP_PROP_FPS)
    dataset_name = video_path.stem
    video_nbr_frame = int(video_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print('Video open.')

Video open.


## Parameters and folder creation

In [5]:
dataset_path = datasets_path / dataset_name
extraction_fps = 3
extraction_resolution = 1024

if (dataset_path).is_dir():
    if len([x for x in dataset_path.glob('**/*')]) > 0:
        print('\N{Heavy Exclamation Mark Symbol} Dataset already created \N{Heavy Exclamation Mark Symbol}')
        print('Delete current dataset folder ({}) to regenerate images.'.format(dataset_path))
else:
    %mkdir "{dataset_path}"

In [6]:
frame_counter_out = 0

for counter in tqdm(range(video_nbr_frame)):
    if (video_cap.isOpened()):
        ret, frame = video_cap.read()

        if counter % int(video_fps / extraction_fps) == 0:
            frame_counter_out += 1
            frame_squared = cv2.resize(frame, (extraction_resolution,extraction_resolution), interpolation = cv2.INTER_AREA)
            cv2.imwrite(str(dataset_path / (dataset_name + '_{}.jpg'.format(frame_counter_out))), frame_squared)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    else:
        break

100%|██████████| 29046/29046 [11:35<00:00, 41.75it/s]


In [7]:
video_cap.release()
cv2.destroyAllWindows()

## Check dataset

In [8]:
for path in dataset_path.glob('**/*.jpg'):
    path_str = str(path)
    img = cv2.imread(path_str)
    dimensions = img.shape
    if dimensions != (1024,1024,3):
        print('Error')